In [1]:
import subprocess

# Startet `ollama serve` im Hintergrund
process = subprocess.Popen(["ollama", "serve"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
print("Ollama server is now running in the background.") 

Ollama server is now running in the background.


In [2]:
# Lege das LLM fest, mit dem nachfolgend gearbeitet wird.
model = "qwen2.5:7b"
model_custom = "custom_qwen2.5:7b_seed_42_temp_0"

In [3]:
# Code to create a model with both seed and temperature set in the Modelfile

# Define the content of the Modelfile with a specified model, seed, and temperature
seed_value = 42  # Example seed value for reproducible results
temperature_value = 0  # Setting temperature to 0 for deterministic output

# Modelfile content with seed and temperature parameters
modelfile_content = f"FROM {model}\nPARAMETER seed {seed_value}\nPARAMETER temperature {temperature_value}\n"

# Write the content to a Modelfile
modelfile_path = "Modelfile"  # Path to save the Modelfile in the current directory

with open(modelfile_path, "w") as modelfile:
    modelfile.write(modelfile_content)

# Command to create the model using the modified Modelfile
command = ["ollama", "create", "custom_qwen2.5:7b_seed_42_temp_0", "-f", modelfile_path]

# Run the command to create the model
try:
    result = subprocess.run(command, capture_output=True, text=True, check=True)
    print(result.stdout)  # Print standard output if successful
except subprocess.CalledProcessError as e:
    print("An error occurred:", e.stderr)  # Display any error messages if the command fails
except FileNotFoundError:
    print("ERROR: 'ollama' command not found. Ensure that the Ollama CLI is installed and accessible.")


In [4]:
import subprocess

# Befehl als Liste definieren
command = ["ollama", "create", "custom_qwen2.5:7b_seed_42", "-f", "Modelfile"]

# Befehl ausführen und Ausgabe anzeigen
try:
    result = subprocess.run(command, capture_output=True, text=True, check=True)
    print(result.stdout)  # Gibt die Standardausgabe aus, falls erfolgreich
except subprocess.CalledProcessError as e:
    print("Ein Fehler ist aufgetreten:", e.stderr)  # Zeigt eventuelle Fehlermeldungen an
except FileNotFoundError:
    print("FEHLER: 'ollama'-Befehl wurde nicht gefunden. Bitte sicherstellen, dass die Ollama-CLI installiert und im Pfad verfügbar ist.")


In [5]:
# Gewähltes Modell von Ollama herunterladen
process = subprocess.Popen(["ollama", "pull", model], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Ausgabe anzeigen
stdout, stderr = process.communicate()
print(stdout.decode("utf-8"))

if stderr:
    print("Error:", stderr.decode("utf-8"))
else:
    print(f"{model} pulled successfully.")


Error: pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest 
pulling 2bada8a74506... 100% ▕████████████████▏ 4.7 GB                         
pulling 66b9ea09bd5b... 100% ▕████████████████▏   68 B                         
pulling eb4402837c78... 100% ▕████████████████▏ 1.5 KB                         
pulling 832dd9e00a68... 100% ▕████████████████▏  11 KB                         
pulling 2f15b3218f05... 100% ▕████████████████▏  487 B                         
verifying sha256 digest 
writing manifest 
success 



In [6]:
import faker
import pandas as pd
import random
from faker import Faker
from datetime import datetime, timedelta
import os

# Seed setzen für Reproduzierbarkeit
random.seed(42)
Faker.seed(42)
fake = Faker()

# Ordner 'tables' erstellen, falls nicht vorhanden
output_folder = "tables"
os.makedirs(output_folder, exist_ok=True)

# Funktion, um zufällige Datumswerte zu erzeugen
def random_date(start, end):
    return start + timedelta(days=random.randint(0, (end - start).days))

# Anzahl der Datensätze
num_students = 100
num_courses = 10
num_professors = 5
num_departments = 3
num_enrollments = 300

# 1. STUDENT_DIMENSION - Generierung der Studentendaten
student_data = {
    "Student_ID": range(1, num_students + 1),
    "First_Name": [fake.first_name() for _ in range(num_students)],
    "Last_Name": [fake.last_name() for _ in range(num_students)],
    "Date_of_Birth": [random_date(datetime(1990, 1, 1), datetime(2002, 12, 31)).date() for _ in range(num_students)],
    "Enrollment_Date": [random_date(datetime(2020, 1, 1), datetime(2023, 12, 31)).date() for _ in range(num_students)]
}

# Erstellung des DataFrames
student_df = pd.DataFrame(student_data)

# E-Mail-Adresse basierend auf First_Name und Last_Name generieren; wenn hier nochmal random verwendet wird, erhalten wir sonst abweichende Namen in der Mailadresse.
student_df["Email"] = student_df["First_Name"].str.lower() + "." + student_df["Last_Name"].str.lower() + "@example.com"

# 2. COURSE_DIMENSION - Generierung der Kursdaten
course_names = [
    "Data Science Basics", "Advanced Machine Learning", "Database Systems",
    "Statistics for Data Science", "Programming with Python", "Ethics in AI",
    "Big Data Analysis", "Data Visualization", "Project Management", "Deep Learning"
]
course_data = {
    "Course_ID": range(1, num_courses + 1),
    "Course_Name": course_names,
    "Credits": [random.choice([3, 4, 5]) for _ in range(num_courses)],
    "Department_ID": [random.randint(1, num_departments) for _ in range(num_courses)]
}
course_df = pd.DataFrame(course_data)

# 3. PROFESSOR_DIMENSION - Generierung der Professorendaten
professor_data = {
    "Professor_ID": range(1, num_professors + 1),
    "First_Name": [fake.first_name() for _ in range(num_professors)],
    "Last_Name": [fake.last_name() for _ in range(num_professors)],
    "Email": [f"{fake.first_name().lower()}.{fake.last_name().lower()}@university.com" for _ in range(num_professors)]
}
professor_df = pd.DataFrame(professor_data)

# 4. DEPARTMENT_DIMENSION - Generierung der Abteilungsdaten
department_names = ["Computer Science", "Business Administration", "Psychology"]
department_data = {
    "Department_ID": range(1, num_departments + 1),
    "Department_Name": department_names
}
department_df = pd.DataFrame(department_data)

# 5. ENROLLMENT_FACTS - Generierung der Einschreibungen
enrollment_data = {
    "Enrollment_ID": range(1, num_enrollments + 1),
    "Student_ID": [random.randint(1, num_students) for _ in range(num_enrollments)],
    "Course_ID": [random.randint(1, num_courses) for _ in range(num_enrollments)],
    "Professor_ID": [random.randint(1, num_professors) for _ in range(num_enrollments)],
    "Enrollment_Date": [random_date(datetime(2021, 1, 1), datetime(2023, 12, 31)).date() for _ in range(num_enrollments)],
    "Grade": [random.choice(['A', 'B', 'C', 'D', 'F']) for _ in range(num_enrollments)]
}
enrollment_df = pd.DataFrame(enrollment_data)

# Speichern der DataFrames als CSV-Dateien im 'tables'-Ordner
student_df.to_csv(os.path.join(output_folder, "STUDENT_DIMENSION.csv"), index=False)
course_df.to_csv(os.path.join(output_folder, "COURSE_DIMENSION.csv"), index=False)
professor_df.to_csv(os.path.join(output_folder, "PROFESSOR_DIMENSION.csv"), index=False)
department_df.to_csv(os.path.join(output_folder, "DEPARTMENT_DIMENSION.csv"), index=False)
enrollment_df.to_csv(os.path.join(output_folder, "ENROLLMENT_FACTS.csv"), index=False)

print("CSV-Dateien wurden erfolgreich im 'tables'-Ordner gespeichert.")


CSV-Dateien wurden erfolgreich im 'tables'-Ordner gespeichert.


In [7]:
# Namen für die DataFrames zur besseren Darstellung
dataframe_names = ["STUDENT_DIMENSION", "COURSE_DIMENSION", "PROFESSOR_DIMENSION", "DEPARTMENT_DIMENSION", "ENROLLMENT_FACTS"]

list_of_dataframes = [student_df, course_df, professor_df, department_df, enrollment_df]

# Ausgabe jeder Tabelle als Markdown-Tabelle
for name, df in zip(dataframe_names, list_of_dataframes):
    print(f"\n {name}\n")
    print(df.head().to_markdown(index=False))  # Ausgabe der ersten fünf Zeilen jeder Tabelle im Markdown-Format
    print("\n")  # Leerzeile für bessere Lesbarkeit



 STUDENT_DIMENSION

|   Student_ID | First_Name   | Last_Name   | Date_of_Birth   | Enrollment_Date   | Email                        |
|-------------:|:-------------|:------------|:----------------|:------------------|:-----------------------------|
|            1 | Danielle     | Smith       | 1992-07-01      | 2022-01-11        | danielle.smith@example.com   |
|            2 | Angel        | Le          | 1990-07-24      | 2021-03-25        | angel.le@example.com         |
|            3 | Joshua       | Maldonado   | 1996-03-03      | 2020-10-10        | joshua.maldonado@example.com |
|            4 | Jeffrey      | Herrera     | 1995-06-30      | 2022-11-09        | jeffrey.herrera@example.com  |
|            5 | Jill         | Adams       | 1995-01-03      | 2022-10-07        | jill.adams@example.com       |



 COURSE_DIMENSION

|   Course_ID | Course_Name                 |   Credits |   Department_ID |
|------------:|:----------------------------|----------:|----------------:|


In [8]:
import subprocess

# List available models in Ollama
process = subprocess.Popen(["ollama", "list"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Ausgabe anzeigen
stdout, stderr = process.communicate()
print(stdout.decode("utf-8"))

if stderr:
    print("Error:", stderr.decode("utf-8"))
else:
    print("Available models listed successfully.")

NAME                                ID              SIZE      MODIFIED       
qwen2.5:7b                          845dbda0ea48    4.7 GB    14 seconds ago    
custom_qwen2.5:7b_seed_42           2849624f7fb4    4.7 GB    15 seconds ago    
custom_qwen2.5:7b_seed_42_temp_0    2849624f7fb4    4.7 GB    15 seconds ago    

Available models listed successfully.


In [9]:
from src.variables import database_schema_mermaid

In [10]:
preprompt = "Your task is to generate executable SQL queries based on my provided model. Please adhere strictly to my model. For each of my questions, generate an executable SELECT SQL query. Do not provide any comments or explanations, only the query. Give me only one query per Question, make sure to start the query with 'SELECT' and end it with a ';'. I am using a sqllite-Database."

In [11]:
fachfrage = "Welche Studenten haben am häufigsten die Note 'A' erhalten? Gib mir eine absteigend sortierte Liste der Namen und Anzahl für die Top 10." # input("Bitte gib eine Frage ein, für die eine SQL-Abfrage generiert werden soll: ")

In [26]:
prompt = f"{database_schema_mermaid} {preprompt} {fachfrage}"

In [25]:
import subprocess

# Startet `ollama run` und übergibt den Prompt über stdin
process = subprocess.Popen(
    ["ollama", "run", model_custom],
    stdin=subprocess.PIPE,
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE)

# Senden des Prompts über stdin und Warten auf die Antwort
stdout, stderr = process.communicate(input=prompt.encode())

# Ausgabe der Antwort und eventuelle Fehler anzeigen
print(stdout.decode().strip())
if stderr:
    print("ERROR:", stderr.decode().strip())

I understand you're feeling disappointed with Mr. Schlieter regarding your study project, but perhaps trying to communicate directly or seeking feedback could help improve your perspective and relationship.
ERROR: ⠙ 


In [14]:
from src.functions import extract_sql_statement

In [15]:
# Funktion, die den Output des Modells so trimmt, dass nur das SQL-Statement stehen bleibt.
from src.functions import extract_sql_statement

# Beispielaufruf
output = stdout.decode()

# Anwenden der Extraktionsfunktion
trimmed_output = extract_sql_statement(output)
print(trimmed_output)

No SQL statement found.


In [16]:
import sqlite3
import pandas as pd

In [17]:
from src.functions import create_tables

# Datenbank und Tabellen erstellen
create_tables('university_database.db')

Tabellen wurden erfolgreich erstellt.


In [18]:
# Verbindung und Cursor erstellen
conn = sqlite3.connect('university_database.db')
cursor = conn.cursor()

In [19]:
# Zeige alle Tabellen in der Datenbank an
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('COURSE_DIMENSION',), ('DEPARTMENT_DIMENSION',), ('ENROLLMENT_FACTS',), ('PROFESSOR_DIMENSION',), ('STUDENT_DIMENSION',)]


In [20]:
from src.functions import import_csv_to_sqlite

In [21]:
from src.functions import import_all_csv_to_sqlite

In [22]:
# Beispielaufruf für den Ordner "tables" und eine persistente Datenbank
import_all_csv_to_sqlite('tables', db_name='university_database.db')

Importiert: COURSE_DIMENSION
Tabelle 'COURSE_DIMENSION' wurde aus 'COURSE_DIMENSION.csv' importiert.
Importiert: DEPARTMENT_DIMENSION
Tabelle 'DEPARTMENT_DIMENSION' wurde aus 'DEPARTMENT_DIMENSION.csv' importiert.
Importiert: ENROLLMENT_FACTS
Tabelle 'ENROLLMENT_FACTS' wurde aus 'ENROLLMENT_FACTS.csv' importiert.
Importiert: PROFESSOR_DIMENSION
Tabelle 'PROFESSOR_DIMENSION' wurde aus 'PROFESSOR_DIMENSION.csv' importiert.
Importiert: STUDENT_DIMENSION
Tabelle 'STUDENT_DIMENSION' wurde aus 'STUDENT_DIMENSION.csv' importiert.


In [23]:
import pandas as pd

# Führe die Abfrage aus
cursor.execute(trimmed_output)

print('Das Ergebnis der generierten SQL-Abfrage lautet:\n')

# Lade die Daten in ein DataFrame
df = pd.DataFrame(cursor.fetchall(), columns=[description[0] for description in cursor.description])

# DataFrame anzeigen
print(df)
print(f'\n\nDie hierfür generierte SQL-Abfrage lautet:\n\n{trimmed_output}')


OperationalError: near "No": syntax error